In [1]:
# Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. 
# Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

import pandas
features = pandas.read_csv('./features.csv', index_col='match_id')
x = features.iloc[:,:-6]
x.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [2]:
# Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает
# число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и
# попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

propusk = len(x) - x.count()
propusk[propusk > 0]

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64

In [3]:
# Замените пропуски на нули с помощью функции fillna().

x = x.fillna(0)
x.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [4]:
# Какой столбец содержит целевую переменную? Запишите его название.

y = features['radiant_win']
y.head()


match_id
0    1
1    1
2    0
3    0
4    0
Name: radiant_win, dtype: int64

In [5]:
# Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями 
# на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), 
# не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, 
# и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. 
# Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, 
# попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества 
# деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра 
# n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

from sklearn import model_selection
import sklearn.ensemble
import sklearn.metrics
import time
from sklearn.model_selection import KFold

sgib = sklearn.model_selection.KFold(n_splits=5, random_state=1, shuffle=True)
for estimator in (10,20,30):
    clf = sklearn.ensemble.GradientBoostingClassifier(n_estimators=estimator, random_state=1)
    nachalo = time.time()
    otvet = sklearn.model_selection.cross_val_score(clf, X=x, y=y, cv=sgib, scoring='roc_auc')
    print("Derevo = {}, srednee = {}, vremya = {}".format(estimator, otvet.mean(), time.time()-nachalo))

Derevo = 10, srednee = 0.66483292280491, vremya = 54.134560108184814
Derevo = 20, srednee = 0.6821140369500348, vremya = 107.19000029563904
Derevo = 30, srednee = 0.6896947542059906, vremya = 160.65721249580383


### Отчёт

1 Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
2 Как называется столбец, содержащий целевую переменную?
3 Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.
4 Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

1
first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dire_flying_courier_time/radiant_flying_courier_time (время покупки предмета "flying_courier") - часто команда не покупает "flying_courier" в первые 5 минут
first_blood_time - Событие "первая кровь" редко происходит за первые 5 минут

2
radiant_win

3
160 секунд с качеством 0.689

4
Для достижения большего качества имеет смысл использовать более 30 деревьев. Для ускорения обучения можно уменьшить глубину деревьев.